In [1]:
%matplotlib inline
from __future__ import print_function

try:
    xrange
except NameError:
    xrange = range

import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.decomposition import LatentDirichletAllocation
import lda
import scipy.sparse as sparse
from collections import Counter

<h2>LDAを使ってみよう</h2>

LDAとはトピックモデルの手法の一つで、Pythonでは以下のようなライブラリでLDAを使用することができます。
* scikit-learn (0.17以降）
* lda
* gensim

ちなみに、LDAというと、Linear Discriminant Analysis(線形判別分析）と思う人もいるので、注意（実際、scikit-learnでLDAは線形判別分析と表しており、トピックモデルのLDAはLatentDirichletAllocationという名前）

今回はlda_datasetフォルダにある2つのJSONファイルを使用します。これらはカメリオで収集した2016/10/21の記事をランダムに1000件サンプリングし、形態素解析した結果ですが、少しだけ異なる処理で2種類作成しました。document_word_data.jsonは名詞を全て含んだもの、document_word_data_pnoun.jsonはproper noun、つまり、固有名詞のみのデータです。これら2つを比較するのも面白いです。


<h2>LDAのインプットデータを作ろう</h2>

ここでのゴールは、文書ID x 単語の行列（成分は出現頻度）を作ることです。ここで注意しなければならないのは、この行列は凄まじくスパースである、つまり、ほとんどの成分が0であるということです。0であるということをデータ上に保存するのはもったいないので、0以外の成分のみをデータとして持っておきたいときに役立つのがスパース行列です。

いろいろな表現方法がありますが、ここでは、List of List (LIL）という方法でデータを保存します。
LILはとても簡単です。

例えば、以下のようなAという行列があるとき、

\begin{equation*}
A = \left[
    \begin{array}{rrr}
      0 & 2 & 0 & 0 \\
      1 & 0 & 0 & 1 \\
      0 & 0 & 0 & 1
    \end{array}
  \right]
\end{equation*}

LILは、ゼロではない成分の行と列、その中の値を保存することで、上記の行列の情報を保存します。
なので、上の$A$をLIL形式で表現すると、
\begin{equation*}
(1, 2) = 2 ~~ (1行目2列目の成分が2)\\
(2,1 ) = 1 ~~ (2行目1列目の成分が1)\\
(2,4 ) = 1 ~~ (2行目4列目の成分が1)\\
(3,4 ) = 1 ~~ (3行目4列目の成分が1)\\
\end{equation*}
となります。

では、LDAのインプットデータを作る場合も同様に考えられて、行に文書のインデックス、縦に単語のインデックスをとって、値があるところだけをLIL形式でデータを保存すれば良いということになります。

さて、そこまでやってみましょう。まずデータを読み込みます。

In [2]:
with open("./dataset/document_word_data.json", "r") as f:
    doc_data= json.load(f)

all_doc_index = doc_data.keys()
print("Total Documents: ", len(all_doc_index))

Total Documents:  1000


この読み込んだjsonデータはこんな感じです。

In [3]:
", ".join(doc_data['715'])

'ママ, 子供, 健康, づくり, 新た, ライフスタイル, 提案, ママ, マルシェ, 府, 府, 市, さまざま, 家族, ら, 日, もん, 商品, ほか, ハロ, 子供, 商品, プレゼント, 各日, 人, 会場, 木, ぬくもり, 木, 子供, づくり, 会, 木, 日, 午後, 時, 今年度, 森林, 林業, 木材, 大使, ミス, 日本, みどり, 帆, 南, さん, 府, 木材, 会, 湯川, 昌子, さん, ら, 女性, 人, 参加, スギ, ヒノキ, 木材, 放出, 健康, 効果, 木材, こと, 女性, 入場, 無料, 文化, 園, 入園, 料, 大人, 円, 円'

単語のインデックスを作るために、全ての単語のリストを作ります。

In [4]:
all_vocab = []
for doc_idx in all_doc_index:
    all_vocab += doc_data[doc_idx]

#重複を消すためにsetしてlistにする
all_vocab = list(set(all_vocab))
vocab_num = len(all_vocab)
print("Vocablary Number: ", vocab_num)

Vocablary Number:  8709


LIL形式の行列を定義します。ScipyにはLIL形式でデータを保存するための機能がありますので、そちらを使います（ScipyはLIL以外のスパース行列のフォーマットをサポートしていますが）。

ここからは学習データと学習後に適用するデータ（ここでは便宜的にテストデータ）に分けましょう。
LDAは教師あり機械学習ではないので、ここでのテストデータの役割は学習済みのモデルに当てはめるデモ用という位置付けです。

In [5]:
all_doc_index_ar = np.array(list(all_doc_index))

train_portion = 0.7
train_num = int(len(all_doc_index_ar) * train_portion)

np.random.shuffle(all_doc_index_ar)
train_doc_index = all_doc_index_ar[:train_num]
test_dox_index = all_doc_index_ar[train_num:]

先にからっぽのスパース行列を定義します。

In [6]:
train_A = sparse.lil_matrix((len(train_doc_index), len(all_vocab)), dtype = np.int)
test_A = sparse.lil_matrix((len(test_dox_index), len(all_vocab)), dtype = np.int)

ListからNumpyのArrayに直します。

In [7]:
all_vocab_ar = np.array(all_vocab)
train_doc_index_ar = np.array(train_doc_index)
test_doc_index_ar = np.array(test_dox_index)

スパース行列に成分を入れていきます。

In [8]:
#学習用
train_total_elements_num = 0
for i in xrange(len(train_doc_index)):
    doc_idx = train_doc_index[i]
    row_data = Counter(doc_data[doc_idx])
    
    for word in row_data.keys():
        word_idx = np.where(all_vocab_ar == word)[0][0]
        train_A[i, word_idx] = row_data[word]
        train_total_elements_num += 1
print("Train total elements num :", train_total_elements_num)


#テスト用
test_total_elements_num = 0
for i in xrange(len(test_dox_index)):
    doc_idx = test_dox_index[i]
    row_data = Counter(doc_data[doc_idx])
    
    for word in row_data.keys():
        word_idx = np.where(all_vocab_ar == word)[0][0]
        test_A[i, word_idx] = row_data[word]
        test_total_elements_num += 1
print("Test total elements num :", test_total_elements_num)

Train total elements num : 33609
Test total elements num : 14435


## 実際にLDAを適用してみよう (Scikit-learnを使った例）

ここではsikit-learnの例を示します。scikit-learnのLDAはオンライン変分ベイズ法という推定方法を用いています。オンラインと名前が付いているので勘が良い方ならお気づきだと思いますが、SGDと同じように部分的にフィット(partial fit)させて、捨てるという形での推定が可能、つまり追加で学習がしやすいというところが特徴です。

In [9]:
model1 = LatentDirichletAllocation(n_topics=20, 
                                doc_topic_prior= 0.001,
                               topic_word_prior=0.5,
                                max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [10]:
model1.fit(train_A)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=0.001,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_jobs=1, n_topics=20, perp_tol=0.1, random_state=0,
             topic_word_prior=0.5, total_samples=1000000.0, verbose=0)

まずトピック x 単語を見てみましょう

In [11]:
normalize_components = model1.components_ /model1.components_.sum(axis=0)

In [12]:
#http://scikit-learn.org/stable/auto_examples/applications/topics_extraction_with_nmf_lda.html　より
n_top_words = 20
for topic_idx, topic in enumerate(normalize_components):
    print("Topic #%d:" % topic_idx)
    print(" ".join([all_vocab_ar[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

Topic #0:
棒 ブラ 大川 ホック 始末 はさみ ブック カラダ バンド イグアイン ミラン がま口 たわし 左利き セリエ 交差 つ折り 人差し指 フェルト メイド

Topic #1:
ダンベル ジム マシン 器具 島根 スタイラス マグネット キタンクラブ 入り口 徳島 ネオジウム 収束 兵庫 マイクロリッジ 南東 ウェイト チェストプレス 栃 日吉津 ウェイトベルト

Topic #2:
日 月 こと 年 人 の よう 円 時 日本 ため 方 分 東京 女性 さん ん イベント 情報 市

Topic #3:
和南 東大和 チェア アッフル マカロン 清瀬 帝京 債 東大 実 準決勝 ボンド 円建て 債券 ペカ ジュ ジェイクライプ アイドリング 周波 アイアンスヘ

Topic #4:
なつ トリガ 本土 セフ 東京大学 俊彦 崩れ わら 沖 将希 ガブガブ かさ キムキョンジャ 水嶋 スロアプリ まさし スリップ 干支 お母様 ルフフェア

Topic #5:
ポグバ ユナイテッド ラプソリュ ランコム 倦怠期 ナウ 加入 センシュアル エンブレム 振替 マット マルカ 今夏 ユベントス オレンジ フェミニン 仕掛け マッチ 意地 ゲンダイ

Topic #6:
マリオピカチュウ 朝比奈 南方 マスコット 海南 マリオ グッズ ピカチュウ オリジナルグッズ 前期 ぬいぐるみ 広州 マリオピカチュウスペシャル ちょ サロペット アモイ チャイナ ヒゲ 動揺 エア

Topic #7:
海津 亮介 ヴィツェル ゼニト ヒルナンデス パン ごっこ マスク シアワセ クロワッサン ヴァ 夏希 導 モンハン ドラクエコラボ クソワロタ ふみ パズドラ ロケ かあさん

Topic #8:
ブラシ ブラッシング 毛 抜け毛 ツヤ バルセロナ 就寝 埃 汚れ ナイロン 入浴 弾力 ただ メンズ クッション 加減 巧 水洗 バルサ タイミング

Topic #9:
曽根崎新地 成徳 檜山 少数 ト 可否 てん リック しそ ヘルス サン ブイヨン ニヨン 安易 東北大 おっぱい お母様 ドイル 浩文 エフ

Topic #10:
栗 小布施 モンブラン ハウス 宿 小布施堂 山梨 テラス 庵 本店 あん ジャングル 栗子 ホタル チェンマイ 液晶 フィルム 焼き

文書 x トピック行列側も見てみましょう。

In [13]:
doc_topic_data = model1.transform(train_A)
doc_topic_data

array([[  4.92562309e-06,   4.92562309e-06,   6.32897504e-01, ...,
          4.92562309e-06,   4.92562309e-06,   4.92562309e-06],
       [  4.75737393e-05,   4.75737393e-05,   9.99096099e-01, ...,
          4.75737393e-05,   4.75737393e-05,   4.75737393e-05],
       [  3.56887937e-05,   3.56887937e-05,   5.40818685e-01, ...,
          3.56887937e-05,   3.56887937e-05,   3.56887937e-05],
       ..., 
       [  1.85116623e-05,   1.85116623e-05,   9.99648278e-01, ...,
          1.85116623e-05,   1.85116623e-05,   1.85116623e-05],
       [  4.44404942e-06,   4.44404942e-06,   9.99915563e-01, ...,
          4.44404942e-06,   4.44404942e-06,   4.44404942e-06],
       [  2.32450023e-05,   2.32450023e-05,   9.99558345e-01, ...,
          2.32450023e-05,   2.32450023e-05,   2.32450023e-05]])

scikit-learnのLDAはどうやら正規化されていないため、正規化した上で、1つ目の文書がどのトピックから来ている単語が多いのかを見てみましょう。

In [14]:
normalize_doc_topic_data = doc_topic_data/doc_topic_data.sum(axis=1, keepdims=True)

In [15]:
for topic_idx, prob in enumerate(normalize_doc_topic_data[0]):
    print("Topic #%d: probality: %f" % (topic_idx, prob))

Topic #0: probality: 0.000005
Topic #1: probality: 0.000005
Topic #2: probality: 0.632898
Topic #3: probality: 0.000005
Topic #4: probality: 0.000005
Topic #5: probality: 0.000005
Topic #6: probality: 0.000005
Topic #7: probality: 0.000005
Topic #8: probality: 0.000005
Topic #9: probality: 0.000005
Topic #10: probality: 0.000005
Topic #11: probality: 0.000005
Topic #12: probality: 0.000005
Topic #13: probality: 0.000005
Topic #14: probality: 0.000005
Topic #15: probality: 0.367014
Topic #16: probality: 0.000005
Topic #17: probality: 0.000005
Topic #18: probality: 0.000005
Topic #19: probality: 0.000005


当てはまりの度合いを測る指標の1つとして対数尤度(Loglikelihood)があります。できるだけ0に近いほどよく当てはまっていることになりますが、「X以上あれば精度が良い」と言えるような絶対的な指標ではなく、ハイパーパラメーターを変えた時などの相対的な比較に使うものだという点を気をつけてください。

In [16]:
loglikelihood = model1.score(test_A)
ppl = model1.perplexity(test_A)
print("対数尤度: ", loglikelihood)
print("Perplexity: ", ppl)

対数尤度:  -682950.745332
Perplexity:  181288115381.0


テストデータに当てはめてみましょう。

In [17]:
test_doc_topic_data = model1.transform(test_A)
normalize_test_doc_topic_data = test_doc_topic_data/test_doc_topic_data.sum(axis=1, keepdims=True)
for topic_idx, prob in enumerate(normalize_test_doc_topic_data[0]):
    print("Topic #%d: probality: %f" % (topic_idx, prob))

Topic #0: probality: 0.000010
Topic #1: probality: 0.000010
Topic #2: probality: 0.877610
Topic #3: probality: 0.000010
Topic #4: probality: 0.000010
Topic #5: probality: 0.000010
Topic #6: probality: 0.000010
Topic #7: probality: 0.122217
Topic #8: probality: 0.000010
Topic #9: probality: 0.000010
Topic #10: probality: 0.000010
Topic #11: probality: 0.000010
Topic #12: probality: 0.000010
Topic #13: probality: 0.000010
Topic #14: probality: 0.000010
Topic #15: probality: 0.000010
Topic #16: probality: 0.000010
Topic #17: probality: 0.000010
Topic #18: probality: 0.000010
Topic #19: probality: 0.000010


<h2> LDAを適用してみよう (ldaパッケージを使った場合)</h2> 

ldaパッケージはより高速なGibbs samplingという手法を使って推定するパッケージです。ただし、これはオンライン学習はできない、つまりバッチ処理しかできません。また、変分ベイズ法による推定とGibbs samplingのどちらが精度が良いかというのはわかりません。

In [18]:
model2 = lda.LDA(n_topics=20, n_iter=1500, random_state=1, alpha=0.5, eta=0.5)

In [21]:
model2.fit(train_A)

INFO:lda:n_documents: 700
INFO:lda:vocab_size: 8709
INFO:lda:n_words: 59539
INFO:lda:n_topics: 20
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -680872
INFO:lda:<10> log likelihood: -536233
INFO:lda:<20> log likelihood: -504567
INFO:lda:<30> log likelihood: -498322
INFO:lda:<40> log likelihood: -497583
INFO:lda:<50> log likelihood: -497217
INFO:lda:<60> log likelihood: -497090
INFO:lda:<70> log likelihood: -497645
INFO:lda:<80> log likelihood: -496475
INFO:lda:<90> log likelihood: -496952
INFO:lda:<100> log likelihood: -497135
INFO:lda:<110> log likelihood: -496887
INFO:lda:<120> log likelihood: -497318
INFO:lda:<130> log likelihood: -497490
INFO:lda:<140> log likelihood: -496499
INFO:lda:<150> log likelihood: -497342
INFO:lda:<160> log likelihood: -496959
INFO:lda:<170> log likelihood: -497451
INFO:lda:<180> log likelihood: -496253
INFO:lda:<190> log likelihood: -496619
INFO:lda:<200> log likelihood: -497028
INFO:lda:<210> log likelihood: -496397
INFO:lda:<220> log likelihood: -4

In [20]:
topic_word = model2.topic_word_
n_top_words = 20
for topic_idx, topic in enumerate(topic_word):
    print("Topic #%d:" % topic_idx)
    print(" ".join([all_vocab_ar[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

Topic #0:
バイト アルバイト トランプ 在住 入学 先 ミス 就活 合格 みなさん 休業 学業 コツ クリントン 夫妻 法 奨学 対処 挑戦 幼児

Topic #1:
栗 味噌 ジャクソン 明 小布施 味 弓子 ドラマ コラボ スカイ 大倉 塔 マンション ジョンソン 漬け 佐 モンブラン 夜 教室 剛

Topic #2:
企業 化 日本 こと 性 提供 支援 くしゃみ 向け 広告 協会 光 年 法人 ため 対応 ビジネス タイヤ システム 安全

Topic #3:
コナン 探偵 アニメ スタジオ 占い 江 仲 安室 アンジュルム バンド ビュッシュ 剛 新曲 昌 ドラム 暮 体制 アプリ ハロ 法律

Topic #4:
ゴジラ 曲 タイム ライン ホップ アルバム ヒップ バンド ホック ブック トラック シン 大作 ラップ ブラ 浮気 ミ ナナ ラブラブ プリント

Topic #5:
位 年 投手 戦 指名 ドラフト 優勝 日 回 出場 プロ 広島 期待 勝利 日本ハム 点 対戦 勝 手 決勝

Topic #6:
こと の 人 よう さん 年 ため 歳 ん もの 方 円 女性 たち そう これ 日本 日 価格 何

Topic #7:
市 倉吉 弱 岡山 午後 強 区 さ 揺れ 広島 気温 津波 ごろ 北栄 熊本 災害 兵庫 梨 分 湯

Topic #8:
アイテム 大人 柄 ニット バッグ ノルディック おじ 感 冬 海津 ブラシ 毛 カジュアル コチラ 亮介 パンツ 店 夏 ドレス プラス

Topic #9:
声優 所 捜査 士 死亡 動機 授業 巨人 志望 戸塚 平塚 望 ごろ 法 所属 役 古城 傷害 所内 卒業

Topic #10:
すみれ エイリアン グレイ ピット 同性愛 マッサン 人影 火 デジカメ スゴイ トレッサ 提案 先月 次回 このほど 先日 手 めちゃくちゃ 正午 リアル

Topic #11:
商品 サイズ さ 枚 こ す 価格 板 個 作り方 店 円 名 式 インテリア セ タイプ おしゃれ リア 横

Topic #12:
入場 新婦 ハノイ 土曜日 ドン サプライズ 携帯 レスリング 扉 払い戻し 全文 文春 ストライプ ロンドン 府 初日 アディダス 整体 キムタク ラブ

Topic #13:
巻

今回も精度として対数尤度を見てみましょう

In [ ]:
model2.loglikelihood()

文書 x トピック行列を見てみましょう。

In [ ]:
doc_topic_data2 = model2.transform(train_A)
for topic_idx, prob in enumerate(doc_topic_data2[0]):
    print("Topic #%d: probality: %f" % (topic_idx, prob))

<h2>（参考）ディリクレ分布の挙動</h2>

In [24]:
alpha = 0.1
K = 6
sampled_probs = np.random.dirichlet([alpha for i in range(K)])
for i, prob in enumerate(sampled_probs):
    print("サイコロ %d面  確率: %.2f"%(i+1, prob))

サイコロ 1面  確率: 0.00
サイコロ 2面  確率: 0.81
サイコロ 3面  確率: 0.02
サイコロ 4面  確率: 0.00
サイコロ 5面  確率: 0.17
サイコロ 6面  確率: 0.00


<h2>（参考）潜在ディリクレ分配法　少しばかり数式を使った解説</h2>
<p>
ある1つの文書データdの生成過程を数式で書くと,<br><br>
文書データdの単語の個数を$N_d$とします。
<br>
<br>
<h4>1. トピックの箱の選ばれやすさを決める</h4>
トピックの確率分布はディリクレ分布に従うものとします。つまり、$\theta_d \sim Dir(\alpha)$ です。この式の意味は、どのトピックの箱がどの程度選ばれるかが$\theta_d$でハイパーパラメーターが$\alpha$ということになり、$\alpha$が1の時、どのトピックの箱が選ばれやすいかは完全にランダムになります。
<br>
<br>
次に各単語について、以下の手順を想定します。
<h4>2. トピックの箱を決める</h4>
トピックの箱を決めるのにカテゴリ分布（多項分布）を想定します（ディリクレ分布とカテゴリ分布は共役だったことを思い出してください）。つまり、$z_{dn} \sim Category(\theta_d)$　を仮定します（$z_{dn}$はトピックの箱の番号だと思ってください）

<h4>3. 単語カードを引く</h4>
トピックの箱を決めた時に、そのトピックの箱から単語カード$w_n$を引く確率を$P(w_{dn} | \phi_{z_n})$ ($n = 1,2, 3, \cdots,  N_d$)と書きます。そして、この$P(w_dn | \phi_{z_{dn}})$を多項分布、$\phi_{z_dn}$をディリクレ分布で表現します。つまり、
\begin{equation*}
\phi_{z_{dn}} \sim   Dir(\beta)
\\
w_{dn} \sim Category(\phi_{z_{dn}} )
\end{equation*}
となります。
<br>
<br>
上記を組みわせて文書データdの生成仮定を以下のように書くことができます。

\begin{equation*}
P(d | \alpha, \bf \beta ) = \int P(\theta_d | \alpha) \prod_{n=1}^{N_d}P(z_{dn} |\theta_d) P(w_{dn} | \phi_{z_{dn}}) P(\phi_{z_{dn}} | \beta)d\theta_d
\end{equation*}
そして、M個の文書データ全体を$D$とすると以下のようになります。
\begin{equation*}
P(D | \alpha, \bf \beta ) = \prod_{d=1}^{M}\int P(\theta_d | \alpha) \prod_{n=1}^{N_d}P(z_{dn} |\theta_d) P(w_{dn} | \phi_{z_{dn}}) P(\phi_{z_{dn}} |
\end{equation*}

若干、デフォルメしてありますが、上記を模したものが以下です。
まず、以下のようにサッカー、音楽、経済の3つのトピックがあるものと仮定します。そして、それぞれのトピックに対し、どの単語がどの程度でやすいのかをディリクレ分布よりサンプリングします。その結果を「phi_トピック名」という配列に入れます。

In [ ]:
topic_soccer = ["サッカー", "本田圭佑", "セリエA", "日本代表", "香川真司"]
topic_music = ["音楽", "ライブ", "ギター", "コンサート", "最高", "武道館"]
topic_econ = ["経済", "株価", "日本企業", "スタートアップ", "Fintech", "マーケティング"]
topics = [topic_soccer, topic_music, topic_econ]

beta = 0.5
phi_soccer = np.random.dirichlet([beta for i in range(len(topic_soccer))])
phi_music = np.random.dirichlet([beta for i in range(len(topic_music))])
phi_econ = np.random.dirichlet([beta for i in range(len(topic_econ))])

phi = [phi_soccer, phi_music, phi_econ]

print("サッカートピックのPhi: ", phi_soccer)
print("音楽トピックのPhi: ", phi_music)
print("経済トピックのPhi: ", phi_econ)

In [ ]:
N = 10  #単語数
alpha = 0.5 #トピック分布のハイパーパラメータ
K = 3 #トピック数
theta = np.random.dirichlet([alpha for i in range(3)])
print("Theta :", theta)

generate_doc = []

for i in range(N):
    selected_topic_ar = np.random.multinomial(1.0, theta)
    selected_topic_idx = np.where(selected_topic_ar==1)[0][0]
    
    
    selected_word_ar = np.random.multinomial(1, phi[selected_topic_idx])
    selected_word_idx = np.where(selected_word_ar==1)[0][0]
    selected_word = topics[selected_topic_idx][selected_word_idx]
    generate_doc.append(selected_word)
    print("i = ", i, "Selected topic:", selected_topic_idx, "Selected word:", selected_word)

    
print('Finally, generated document is "', " ".join(generate_doc))

このような生成プロセスを経て、我々が目にしている文書が得られていると仮定するわけです。<br>
さて、ではLDAで推定したいものは何でしょうか？それは、各トピックと$\theta$と$\phi$とかなのですが、その推定方法は、主に2つあります。一つは変分ベイズ
法による推定、もう一つはGibbs samplingによる推定です。興味がある方は調べてみてください。